In [1]:
import pickle
import sys
sys.path.append("E:/Qin/wrs")
import wrs.basis.robot_math as rm
import wrs.modeling.collision_model as mcm
import wrs.visualization.panda.world as wd
import wrs.modeling.geometric_model as mgm
import wrs.robot_sim.end_effectors.grippers.wrs_gripper.wrs_gripper_v3 as wrs_gripper_v3
import numpy as np
import matplotlib.pyplot as plt

base = wd.World(cam_pos=[1.7, 1.7, 1.7], lookat_pos=[0, 0, .3])
mgm.gen_frame().attach_to(base)


def grasp_load(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def create_target_vector(label_dim, label_ids):
    target_vector = np.zeros(label_dim, dtype=np.float32)
    if label_ids == None:
        return target_vector
    target_vector[label_ids] = 1
    return target_vector


In [3]:
import seaborn as sns
def analyze_ones_distribution(target_vector_sets, grasp_type_label=""):
    """
    分析不同实验数据的正样本占比
    
    参数:
    target_vector_sets: 字典，键为实验编号，值为对应的目标向量集合
    grasp_type_label: 抓取类型标签
    
    返回:
    统计分析结果字典
    """
    import numpy as np
    
    # 初始化结果存储
    result = {}
    
    # 对每个实验数据集合进行分析
    for exp_num, target_vectors in target_vector_sets.items():
        # 将所有向量合并为一个大向量以计算整体占比
        all_elements = []
        class_distribution = []  # 存储类别分布信息
        for vector in target_vectors:
            all_elements.extend(vector)
            class_distribution.append(vector)
        
        if all_elements:
            # 计算所有向量中1的总体占比
            overall_ratio = np.sum(all_elements) / len(all_elements)
            
            # 存储结果
            result[exp_num] = {
                'overall_ratio': overall_ratio,
                'total_elements': len(all_elements),
                'positive_count': int(np.sum(all_elements)),
                'class_distribution': np.array(class_distribution)  # 添加类别分布信息
            }
    
    return result


def compare_grasp_types(shared_result, feasible_result, exp_numbers):
    """
    绘制Shared Grasp和Feasible Grasp的比较图表
    
    参数:
    shared_result: Shared Grasp的分析结果
    feasible_result: Feasible Grasp的分析结果
    exp_numbers: 实验编号列表
    """
    import numpy as np
    import matplotlib.pyplot as plt
    
    # 设置全局字体为Times New Roman
    plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['mathtext.fontset'] = 'stix'  # 数学公式字体也使用类似Times的字体
    
    # 创建单个图表
    plt.figure(figsize=(12, 6), dpi=800)
    
    # 获取数据
    shared_ratios = [shared_result[num]['overall_ratio'] if num in shared_result else 0 for num in exp_numbers]
    feasible_ratios = [feasible_result[num]['overall_ratio'] if num in feasible_result else 0 for num in exp_numbers]
    
    # 将exp_numbers转为字符串以用于x轴标签
    grasp_ids = [str(num) for num in exp_numbers]
    
    # 绘制折线图
    plt.plot(grasp_ids, shared_ratios, '-o', color='blue', linewidth=4, markersize=20, label='Shared Grasp')
    plt.plot(grasp_ids, feasible_ratios, '-o', color='green', linewidth=4, markersize=20, label='Feasible Grasp')
    
    # 设置图表
    plt.xlabel('Grasp candidate number', fontsize=34)
    plt.ylabel('Positive Sample Ratio', fontsize=34)
    plt.xticks(fontsize=34)
    plt.yticks(fontsize=34)
    
    # 设置网格
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # 调整Y轴范围
    plt.ylim(0, max(max(shared_ratios), max(feasible_ratios)) * 1.1)
    
    # 添加图例
    plt.legend(fontsize=34)
    
    plt.tight_layout()
    plt.savefig('grasp_comparison_with_positive_ratio.png', dpi=1000, bbox_inches='tight')
    plt.show()

In [ ]:

if __name__ == '__main__':
    # 定义要分析的编号列表
    exp_numbers = [57, 83, 109, 352]
    
    # 初始化存储所有实验数据的字典
    shared_grasp_sets = {}  # 存储Shared Grasp数据 (common_grasp_data_item[-1])
    feasible_grasp_sets = {}  # 存储Feasible Grasp数据 (common_grasp_data_item[1])
    
    # 加载每个实验编号对应的数据
    for exp_num in exp_numbers:
        # grasp data
        grasp_data_path = fr"E:\Qin\wrs\wrs\HuGroup_Qin\Shared_grasp_project\grasps\Bottle\bottle_grasp_{exp_num}.pickle"
        object_feasible_grasps = grasp_load(grasp_data_path)
        id_dim = len(object_feasible_grasps)
        
        # common grasp data
        common_grasp_data_path = fr"E:\Qin\wrs\wrs\HuGroup_Qin\Shared_grasp_project\grasps\Bottle\SharedGraspNetwork_bottle_experiment_data_{exp_num}.pickle"
        common_grasp_data = grasp_load(common_grasp_data_path)
        
        # 创建Shared Grasp的目标向量集合
        shared_grasp_vectors = []
        # 创建Feasible Grasp的目标向量集合
        feasible_grasp_vectors = []
        
        for common_grasp_data_index in range(len(common_grasp_data)):
            common_grasp_data_item = common_grasp_data[common_grasp_data_index]
            
            # 创建Shared Grasp的目标向量
            shared_grasp_vectors.append(create_target_vector(id_dim, common_grasp_data_item[-1]))
            
            # 创建Feasible Grasp的目标向量
            feasible_grasp_vectors.append(create_target_vector(id_dim, common_grasp_data_item[1]))
        
        # 存储到对应的字典中
        shared_grasp_sets[exp_num] = shared_grasp_vectors
        feasible_grasp_sets[exp_num] = feasible_grasp_vectors
    
    # 分析两种类型的数据
    shared_result = analyze_ones_distribution(shared_grasp_sets, "Shared Grasp")
    feasible_result = analyze_ones_distribution(feasible_grasp_sets, "Feasible Grasp")
    
    # 绘制比较图表
    compare_grasp_types(shared_result, feasible_result, exp_numbers)